In [1]:
import vastai
import asyncio
import random
import logging

logging.getLogger("Serverless").propagate = False
# or, if the library uses a package logger name:
logging.getLogger("vastai").propagate = False


In [4]:
import asyncio
import random
import vastai

async def start_session():
    client = vastai.Serverless()
    session_a = None
    session_b = None

    try:
        endpoint = await client.get_endpoint("my-comfy-endpoint")

        payload = {
            "input": {
                "modifier": "Text2Image",
                "modifications": {
                    "prompt": "Generate a page from a peanuts comic strip.",
                    "width": 512,
                    "height": 512,
                    "steps": 10,
                    "seed": random.randint(1, 1000)
                }
            }
        }

        session_a = await endpoint.session()
        session_b = await endpoint.session()

        def print_response(response):
            print(response["response"]["output"][0]["local_path"])

        while True:
            await asyncio.gather(
                session_a.request("/generate/sync", payload).then(print_response),
                session_b.request("/generate/sync", payload).then(print_response),
            )

    except asyncio.CancelledError:
        print("Notebook cell cancelled.")
        raise

    finally:
        if session_a:
            await session_a.close()
        if session_b:
            await session_b.close()
        await client.close()
        print("Sessions closed.")


In [7]:
await start_session()

Got response {'response': {'session_id': 'EnVTgPlhKWD13', 'expiration': 1765857448.5177598}, 'latency': 0.41270899772644043, 'url': 'https://199.68.217.31:52109', 'reuqest_idx': 39, 'auth_data': {'__request_id': 'f9420bc5-7d4d-4668-83e5-3ca90a6d4d1b', 'cost': 100.0, 'endpoint': 'my-comfy-endpoint', 'reqnum': 3, 'request_idx': 39, 'signature': 'WHXQAlVxZe+SeXGhokYYxM0luesD/pLSNgHNIz9eH3035PEvnfu4C8s6OO8K3LUOkJ09wEXb5VpqewFWDXsq84aHhPbuyRiMCk6OXOyVjehFq/mtptDBehNYZK7lsyMc8OvJcPs2Jjse53AyoKo6/afSyGWgOGhxwKurcEBZpD439cXtkjhmvHDvKPf+iZ2e40QmlWBjYwAM59WjyVcV0EQ3k4IVOuuPB17A83C9zKZWpBvmtobGut/sBIdKYctI1CY738YAdKMm4Djb59LoGHtIWyR5i5/IfFfTFQbW51S3M6tMyNoBebIb0MNpVeJ1DkkwZl6SPJVSMAO+zP0c+Q==', 'url': 'https://199.68.217.31:52109'}}
Got response {'response': {'session_id': '3AZnP5JnHrksV', 'expiration': 1765857449.018959}, 'latency': 0.42191553115844727, 'url': 'https://199.68.217.31:53822', 'reuqest_idx': 40, 'auth_data': {'__request_id': '734c2cc9-c11f-4dac-a08f-67b176814620', 'cost': 100.0, 'e

CancelledError: 

In [ ]:
import asyncio
import random
import vastai

async def start_training_session():
    client = vastai.Serverless()
    session = None

    try:
        endpoint = await client.get_endpoint("my-pytorch-endpoint")

        payload = {'max_train_batches_per_epoch': 100, "epochs" : 200}

        session = await endpoint.session()

        start_task = await session.request(route="/start_task", payload=payload)
        count = 0
        while True:
            await asyncio.sleep(1)
            status = await session.request(route="/status", payload={})
            print(status)
            if status["response"]["status"]["state"] != "running":
                break
            #if count > 5:
            #    status = await session.request(route="/cancel_task", payload={})
            count += 1
        print("training complete")

    except asyncio.CancelledError:
        print("Notebook cell cancelled.")
        raise

    finally:
        if session:
            await session.close()
        await client.close()
        print("Sessions closed.")


In [9]:
await asyncio.gather(start_training_session())

{'response': {'ok': True, 'status': {'task_id': '0cb23e3c-6799-4821-9d4b-adb741079179', 'state': 'running', 'message': 'Validation epoch 1/200 complete', 'created_at': 1765845162.71553, 'started_at': 1765845162.7155304, 'finished_at': None, 'epoch': 1, 'step': 10, 'total_steps': 2000, 'train_loss': 2.0437816739082337, 'train_acc': 0.365625, 'val_loss': 1.6601170492172241, 'val_acc': 0.5228125, 'last_update_at': 1765845163.627791, 'config': {'epochs': 200, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 0}, 'error_type': None, 'error': None}}, 'latency': 0.1953871250152588, 'url': 'https://79.112.58.103:30111', 'reuqest_idx': 0, 'auth_data': {'__request_id': '61dc3136-e1e9-416d-9d35-8f1f25595dda', 'cost': 100.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 6, 'request_idx': 7, 'signature': 'qrbUeuOV/QmYnGdb03kD1/vGfbWVlf/ptJcBnHuNxj7OnStXxZXG5H0Z/XcpMhEQalPca9SzvEoDO4qVTP4fQozA+U5eYY3i4oFuYUtISqp

[None]